# 0.0 Imports

In [25]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib import gridspec 
import ipywidgets as widgets
from ipywidgets import fixed
import plotly.express as px
import seaborn as sns
from ipywidgets import interact, interactive, fixed, interact_manual

# 1.1 Loading Data

In [2]:
data=pd.read_csv('kc_house_data.csv')

#### 1.Qual a quantidade de imóveis por nível?
     - Nivel 0: Preço entre R$ 0.00 e R$ 321.950
     - Nível 1: Preço entre R$ 321.950 e R$ 450.00
     - Nível 2: Preço entre R$ 450.00 e R$ 645.00
     - Nível 3: Preço acima de R$ 645.00

In [3]:
data['nivel'] = 'NA'
for i in range(len(data)):
    if(data.loc[i, 'price']>0) & (data.loc[i, 'price']< 321950):
        data.loc[i, 'nivel'] = 'nivel_0'
        
    elif(data.loc[i, 'price'] >= 321950) & (data.loc[i, 'price']<450000):
        data.loc[i, 'nivel'] = 'nivel_1'
        
    elif(data.loc[i, 'price']>= 450000) & (data.loc[i, 'price'] < 645000):
        data.loc[i, 'nivel'] = 'nivel_2'

    else: 
        data.loc[i, 'nivel'] = 'nivel_3'
        

#### 2.Adicione as seguintes informações ao imóvel:
- O nome da Rua
- O número do imóvel 
- O nome do Bairro
- O nome da Cidade
- O nome da Estado

In [54]:
# pip install geopy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: geopy in c:\users\dougb\anaconda3\envs\virtualenv\lib\site-packages (2.1.0)



In [4]:
from geopy.geocoders import Nominatim

In [6]:
# Create empty rows
data['road'] = 'NA'
data['house_number'] = 'NA'
data['city'] = 'NA'
data['county'] = 'NA'
data['state'] = 'NA'

geolocator = Nominatim(user_agent = 'geoapiExercises')

for i in range(len(data)):
    print('Loop: {} / {}'.format(i, len(data)))
    # make query
    query = str(data.loc[i, 'lat']) +','+ str(data.loc[i, 'long'])
    
    #API Request
    response = geolocator.reverse(query)
    
    #populate data
    if 'house_number' in response.raw['address']:
        data.loc[i, 'house_number'] = response.raw['address']['house_number']
        
    if 'road' in response.raw['address']:
        data.loc[i, 'road']         = response.raw['address']['road']
        
    if 'neighbourhood' in response.raw['address']:
        data.loc[i, 'neighbourhood']= response.raw['address']['neighbourhood']
        
    if 'city' in response.raw['address']:
        data.loc[i, 'city']         = response.raw['address']['city']
        
    if 'county' in response.raw['address']:
        data.loc[i, 'county']       = response.raw['address']['county']
        
    if 'state' in response.raw['address']:
        data.loc[i, 'state']        = response.raw['address']['state']


Loop: 0 / 21613
Loop: 1 / 21613
Loop: 2 / 21613
Loop: 3 / 21613
Loop: 4 / 21613
Loop: 5 / 21613
Loop: 6 / 21613
Loop: 7 / 21613
Loop: 8 / 21613
Loop: 9 / 21613
Loop: 10 / 21613
Loop: 11 / 21613
Loop: 12 / 21613
Loop: 13 / 21613
Loop: 14 / 21613
Loop: 15 / 21613
Loop: 16 / 21613
Loop: 17 / 21613
Loop: 18 / 21613
Loop: 19 / 21613
Loop: 20 / 21613
Loop: 21 / 21613
Loop: 22 / 21613
Loop: 23 / 21613
Loop: 24 / 21613
Loop: 25 / 21613
Loop: 26 / 21613
Loop: 27 / 21613
Loop: 28 / 21613
Loop: 29 / 21613
Loop: 30 / 21613
Loop: 31 / 21613
Loop: 32 / 21613
Loop: 33 / 21613
Loop: 34 / 21613
Loop: 35 / 21613
Loop: 36 / 21613
Loop: 37 / 21613
Loop: 38 / 21613
Loop: 39 / 21613
Loop: 40 / 21613
Loop: 41 / 21613
Loop: 42 / 21613
Loop: 43 / 21613
Loop: 44 / 21613
Loop: 45 / 21613
Loop: 46 / 21613
Loop: 47 / 21613
Loop: 48 / 21613
Loop: 49 / 21613
Loop: 50 / 21613
Loop: 51 / 21613
Loop: 52 / 21613
Loop: 53 / 21613
Loop: 54 / 21613
Loop: 55 / 21613
Loop: 56 / 21613
Loop: 57 / 21613
Loop: 58 / 21613
Loop: 5

Loop: 462 / 21613
Loop: 463 / 21613
Loop: 464 / 21613
Loop: 465 / 21613
Loop: 466 / 21613
Loop: 467 / 21613
Loop: 468 / 21613
Loop: 469 / 21613
Loop: 470 / 21613
Loop: 471 / 21613
Loop: 472 / 21613
Loop: 473 / 21613
Loop: 474 / 21613
Loop: 475 / 21613
Loop: 476 / 21613
Loop: 477 / 21613
Loop: 478 / 21613
Loop: 479 / 21613
Loop: 480 / 21613
Loop: 481 / 21613
Loop: 482 / 21613
Loop: 483 / 21613
Loop: 484 / 21613
Loop: 485 / 21613
Loop: 486 / 21613
Loop: 487 / 21613
Loop: 488 / 21613
Loop: 489 / 21613
Loop: 490 / 21613
Loop: 491 / 21613
Loop: 492 / 21613
Loop: 493 / 21613
Loop: 494 / 21613
Loop: 495 / 21613
Loop: 496 / 21613
Loop: 497 / 21613
Loop: 498 / 21613
Loop: 499 / 21613
Loop: 500 / 21613
Loop: 501 / 21613
Loop: 502 / 21613
Loop: 503 / 21613
Loop: 504 / 21613
Loop: 505 / 21613
Loop: 506 / 21613
Loop: 507 / 21613
Loop: 508 / 21613
Loop: 509 / 21613
Loop: 510 / 21613
Loop: 511 / 21613
Loop: 512 / 21613
Loop: 513 / 21613
Loop: 514 / 21613
Loop: 515 / 21613
Loop: 516 / 21613
Loop: 517 

Loop: 918 / 21613
Loop: 919 / 21613
Loop: 920 / 21613
Loop: 921 / 21613
Loop: 922 / 21613
Loop: 923 / 21613
Loop: 924 / 21613
Loop: 925 / 21613
Loop: 926 / 21613
Loop: 927 / 21613
Loop: 928 / 21613
Loop: 929 / 21613
Loop: 930 / 21613
Loop: 931 / 21613
Loop: 932 / 21613
Loop: 933 / 21613
Loop: 934 / 21613
Loop: 935 / 21613
Loop: 936 / 21613
Loop: 937 / 21613
Loop: 938 / 21613
Loop: 939 / 21613
Loop: 940 / 21613
Loop: 941 / 21613
Loop: 942 / 21613
Loop: 943 / 21613
Loop: 944 / 21613
Loop: 945 / 21613
Loop: 946 / 21613
Loop: 947 / 21613
Loop: 948 / 21613
Loop: 949 / 21613
Loop: 950 / 21613
Loop: 951 / 21613
Loop: 952 / 21613
Loop: 953 / 21613
Loop: 954 / 21613
Loop: 955 / 21613
Loop: 956 / 21613
Loop: 957 / 21613
Loop: 958 / 21613
Loop: 959 / 21613
Loop: 960 / 21613
Loop: 961 / 21613
Loop: 962 / 21613
Loop: 963 / 21613
Loop: 964 / 21613
Loop: 965 / 21613
Loop: 966 / 21613
Loop: 967 / 21613
Loop: 968 / 21613
Loop: 969 / 21613
Loop: 970 / 21613
Loop: 971 / 21613
Loop: 972 / 21613
Loop: 973 

Loop: 1354 / 21613
Loop: 1355 / 21613
Loop: 1356 / 21613
Loop: 1357 / 21613
Loop: 1358 / 21613
Loop: 1359 / 21613
Loop: 1360 / 21613
Loop: 1361 / 21613
Loop: 1362 / 21613
Loop: 1363 / 21613
Loop: 1364 / 21613
Loop: 1365 / 21613
Loop: 1366 / 21613
Loop: 1367 / 21613
Loop: 1368 / 21613
Loop: 1369 / 21613
Loop: 1370 / 21613
Loop: 1371 / 21613
Loop: 1372 / 21613
Loop: 1373 / 21613
Loop: 1374 / 21613
Loop: 1375 / 21613
Loop: 1376 / 21613
Loop: 1377 / 21613
Loop: 1378 / 21613
Loop: 1379 / 21613
Loop: 1380 / 21613
Loop: 1381 / 21613
Loop: 1382 / 21613
Loop: 1383 / 21613
Loop: 1384 / 21613
Loop: 1385 / 21613
Loop: 1386 / 21613
Loop: 1387 / 21613
Loop: 1388 / 21613
Loop: 1389 / 21613
Loop: 1390 / 21613
Loop: 1391 / 21613
Loop: 1392 / 21613
Loop: 1393 / 21613
Loop: 1394 / 21613
Loop: 1395 / 21613
Loop: 1396 / 21613
Loop: 1397 / 21613
Loop: 1398 / 21613
Loop: 1399 / 21613
Loop: 1400 / 21613
Loop: 1401 / 21613
Loop: 1402 / 21613
Loop: 1403 / 21613
Loop: 1404 / 21613
Loop: 1405 / 21613
Loop: 1406 /

Loop: 1786 / 21613
Loop: 1787 / 21613
Loop: 1788 / 21613
Loop: 1789 / 21613
Loop: 1790 / 21613
Loop: 1791 / 21613
Loop: 1792 / 21613
Loop: 1793 / 21613
Loop: 1794 / 21613
Loop: 1795 / 21613
Loop: 1796 / 21613
Loop: 1797 / 21613
Loop: 1798 / 21613
Loop: 1799 / 21613
Loop: 1800 / 21613
Loop: 1801 / 21613
Loop: 1802 / 21613
Loop: 1803 / 21613
Loop: 1804 / 21613
Loop: 1805 / 21613
Loop: 1806 / 21613
Loop: 1807 / 21613
Loop: 1808 / 21613
Loop: 1809 / 21613
Loop: 1810 / 21613
Loop: 1811 / 21613
Loop: 1812 / 21613
Loop: 1813 / 21613
Loop: 1814 / 21613
Loop: 1815 / 21613
Loop: 1816 / 21613
Loop: 1817 / 21613
Loop: 1818 / 21613
Loop: 1819 / 21613
Loop: 1820 / 21613
Loop: 1821 / 21613
Loop: 1822 / 21613
Loop: 1823 / 21613
Loop: 1824 / 21613
Loop: 1825 / 21613
Loop: 1826 / 21613
Loop: 1827 / 21613
Loop: 1828 / 21613
Loop: 1829 / 21613
Loop: 1830 / 21613
Loop: 1831 / 21613
Loop: 1832 / 21613
Loop: 1833 / 21613
Loop: 1834 / 21613
Loop: 1835 / 21613
Loop: 1836 / 21613
Loop: 1837 / 21613
Loop: 1838 /

...

Loop: 2650 / 21613
Loop: 2651 / 21613
Loop: 2652 / 21613
Loop: 2653 / 21613
Loop: 2654 / 21613
Loop: 2655 / 21613
Loop: 2656 / 21613
Loop: 2657 / 21613
Loop: 2658 / 21613
Loop: 2659 / 21613
Loop: 2660 / 21613
Loop: 2661 / 21613
Loop: 2662 / 21613
Loop: 2663 / 21613
Loop: 2664 / 21613
Loop: 2665 / 21613
Loop: 2666 / 21613
Loop: 2667 / 21613
Loop: 2668 / 21613
Loop: 2669 / 21613
Loop: 2670 / 21613
Loop: 2671 / 21613
Loop: 2672 / 21613
Loop: 2673 / 21613
Loop: 2674 / 21613
Loop: 2675 / 21613
Loop: 2676 / 21613
Loop: 2677 / 21613
Loop: 2678 / 21613
Loop: 2679 / 21613
Loop: 2680 / 21613
Loop: 2681 / 21613
Loop: 2682 / 21613
Loop: 2683 / 21613
Loop: 2684 / 21613
Loop: 2685 / 21613
Loop: 2686 / 21613
Loop: 2687 / 21613
Loop: 2688 / 21613
Loop: 2689 / 21613
Loop: 2690 / 21613
Loop: 2691 / 21613
Loop: 2692 / 21613
Loop: 2693 / 21613
Loop: 2694 / 21613
Loop: 2695 / 21613
Loop: 2696 / 21613
Loop: 2697 / 21613
Loop: 2698 / 21613
Loop: 2699 / 21613
Loop: 2700 / 21613
Loop: 2701 / 21613
Loop: 2702 /

Loop: 3082 / 21613
Loop: 3083 / 21613
Loop: 3084 / 21613
Loop: 3085 / 21613
Loop: 3086 / 21613
Loop: 3087 / 21613
Loop: 3088 / 21613
Loop: 3089 / 21613
Loop: 3090 / 21613
Loop: 3091 / 21613
Loop: 3092 / 21613
Loop: 3093 / 21613
Loop: 3094 / 21613
Loop: 3095 / 21613
Loop: 3096 / 21613
Loop: 3097 / 21613
Loop: 3098 / 21613
Loop: 3099 / 21613
Loop: 3100 / 21613
Loop: 3101 / 21613
Loop: 3102 / 21613
Loop: 3103 / 21613
Loop: 3104 / 21613
Loop: 3105 / 21613
Loop: 3106 / 21613
Loop: 3107 / 21613
Loop: 3108 / 21613
Loop: 3109 / 21613
Loop: 3110 / 21613
Loop: 3111 / 21613
Loop: 3112 / 21613
Loop: 3113 / 21613
Loop: 3114 / 21613
Loop: 3115 / 21613
Loop: 3116 / 21613
Loop: 3117 / 21613
Loop: 3118 / 21613
Loop: 3119 / 21613
Loop: 3120 / 21613
Loop: 3121 / 21613
Loop: 3122 / 21613
Loop: 3123 / 21613
Loop: 3124 / 21613
Loop: 3125 / 21613
Loop: 3126 / 21613
Loop: 3127 / 21613
Loop: 3128 / 21613
Loop: 3129 / 21613
Loop: 3130 / 21613
Loop: 3131 / 21613
Loop: 3132 / 21613
Loop: 3133 / 21613
Loop: 3134 /

Loop: 3514 / 21613
Loop: 3515 / 21613
Loop: 3516 / 21613
Loop: 3517 / 21613
Loop: 3518 / 21613
Loop: 3519 / 21613
Loop: 3520 / 21613
Loop: 3521 / 21613
Loop: 3522 / 21613
Loop: 3523 / 21613
Loop: 3524 / 21613
Loop: 3525 / 21613
Loop: 3526 / 21613
Loop: 3527 / 21613
Loop: 3528 / 21613
Loop: 3529 / 21613
Loop: 3530 / 21613
Loop: 3531 / 21613
Loop: 3532 / 21613
Loop: 3533 / 21613
Loop: 3534 / 21613
Loop: 3535 / 21613
Loop: 3536 / 21613
Loop: 3537 / 21613
Loop: 3538 / 21613
Loop: 3539 / 21613
Loop: 3540 / 21613
Loop: 3541 / 21613
Loop: 3542 / 21613
Loop: 3543 / 21613
Loop: 3544 / 21613
Loop: 3545 / 21613
Loop: 3546 / 21613
Loop: 3547 / 21613
Loop: 3548 / 21613
Loop: 3549 / 21613
Loop: 3550 / 21613
Loop: 3551 / 21613
Loop: 3552 / 21613
Loop: 3553 / 21613
Loop: 3554 / 21613
Loop: 3555 / 21613
Loop: 3556 / 21613
Loop: 3557 / 21613
Loop: 3558 / 21613
Loop: 3559 / 21613
Loop: 3560 / 21613
Loop: 3561 / 21613
Loop: 3562 / 21613
Loop: 3563 / 21613
Loop: 3564 / 21613
Loop: 3565 / 21613
Loop: 3566 /

Loop: 3946 / 21613
Loop: 3947 / 21613
Loop: 3948 / 21613
Loop: 3949 / 21613
Loop: 3950 / 21613
Loop: 3951 / 21613
Loop: 3952 / 21613
Loop: 3953 / 21613
Loop: 3954 / 21613
Loop: 3955 / 21613
Loop: 3956 / 21613
Loop: 3957 / 21613
Loop: 3958 / 21613
Loop: 3959 / 21613
Loop: 3960 / 21613
Loop: 3961 / 21613
Loop: 3962 / 21613
Loop: 3963 / 21613
Loop: 3964 / 21613
Loop: 3965 / 21613
Loop: 3966 / 21613
Loop: 3967 / 21613
Loop: 3968 / 21613
Loop: 3969 / 21613
Loop: 3970 / 21613
Loop: 3971 / 21613
Loop: 3972 / 21613
Loop: 3973 / 21613
Loop: 3974 / 21613
Loop: 3975 / 21613
Loop: 3976 / 21613
Loop: 3977 / 21613
Loop: 3978 / 21613
Loop: 3979 / 21613
Loop: 3980 / 21613
Loop: 3981 / 21613
Loop: 3982 / 21613
Loop: 3983 / 21613
Loop: 3984 / 21613
Loop: 3985 / 21613
Loop: 3986 / 21613
Loop: 3987 / 21613
Loop: 3988 / 21613
Loop: 3989 / 21613
Loop: 3990 / 21613
Loop: 3991 / 21613
Loop: 3992 / 21613
Loop: 3993 / 21613
Loop: 3994 / 21613
Loop: 3995 / 21613
Loop: 3996 / 21613
Loop: 3997 / 21613
Loop: 3998 /

KeyboardInterrupt: 

#### 3.Adicione o nível do imóvel no mapa como uma cor

In [19]:
import plotly.express as px

houses = data[['id', 'lat', 'long', 'price']].copy()

# define level
for i in range(len(houses)):
    if houses.loc[i, 'price'] <= 321950:
        houses.loc[i, 'level'] = 0
        
    elif (houses.loc[i, 'price'] > 321950) & (houses.loc[i, 'price'] <= 450000):
        houses.loc[i, 'level'] = 1
    
    elif (houses.loc[i, 'price'] > 450000) & (houses.loc[i, 'price'] <= 645000):
        houses.loc[i, 'level'] = 2
        
    else:
        houses.loc[i, 'level'] = 3
        
houses['level'] = houses['level'].astype(int)

fig = px.scatter_mapbox( houses,
                        lat = 'lat',
                        lon = 'long',
                        color = 'level',
                        size = 'price',
                        color_continuous_scale = px.colors.cyclical.IceFire,
                        size_max = 15,
                        zoom = 10)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(height = 600, margin = {'r': 0, 't': 0, 'l': 0, 'b':0})
fig.show()




##### 4. Adicione o preço do imóvel como o tamanho do ponto do mapa

#### 5. Adicone opçoes de filtros para eu fazer minhas próprias análises:
1. Eu quero escolher visualizar imóveis com vista para água ou não
2. Eu quero escolher filtrar os imóveis até um certo valor de preço.

In [20]:
# pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [21]:
import ipywidgets as widgets
from ipywidgets import fixed

df = pd.read_csv('kc_house_data.csv')

df['is_waterfront'] = df['waterfront'].apply(lambda x: 'yes' if x==1 else 'no')

df['level'] = df['price'].apply(lambda x: 0 if x < 321950 else
                                           1 if (x>321950) & (x < 450000) else
                                           2 if (x > 450000) & (x < 645000) else 3)

df['level'] = df['level'].astype(int)
style = {'description_with': 'initial'}

# Iterative buttons

price_limit = widgets.IntSlider(
    value = 540000,
    min = 75000,
    max = 77000000,
    step = 1, 
    description = 'Maximun Price',
    disable = False,
    style = style
)


waterfront_bar = widgets.Dropdown(
    options = df['is_waterfront'].unique().tolist(),
    value = 'yes',
    description = 'Water View',
    disable = False)




def update_map(df, waterfront, limit):
    houses = df[(df['price']<= limit) & (df['is_waterfront']==waterfront)][['id', 'lat', 'long', 'price', 'level']]
    
    fig = px.scatter_mapbox(houses,
                           lat = 'lat',
                           lon = 'long',
                           color = 'level',
                           size = 'price',
                           color_continuous_scale = px.colors.cyclical.IceFire,
                           size_max = 15,
                           zoom = 10)
    fig.update_layout(mapbox_style = 'open-street-map')
    fig.update_layout(height = 600, margin = {'r':0, 't':0, 'l':0, 'b': 0})
    fig.show()





In [22]:
widgets.interactive(update_map, df=fixed(df), waterfront = waterfront_bar, limit=price_limit)

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

#### 6.Adicionar opções de filtros no último dashaboard enviado:
1. Eu quero visualizar somente valores a partir de uma data disponível para comprar.

In [24]:
# Prepare dataset
data = pd.read_csv('kc_house_data.csv')

# Change data format
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

#Widgets to control data
date_limit = widgets.SelectionSlider(
    options = data['date'].sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Disponivel',
    continuous_update = False,
    orientation = 'horizontal',
    readout= True)

def update_map(data, limit):
    #Filter data
    df = data[data['date'] >= limit].copy()
    
    fig = plt.figure(figsize = (21,12))
    specs = gridspec.GridSpec(ncols = 2, nrows = 2, figure = fig)
    
    ax1 = fig.add_subplot(specs[0,:]) # first rows
    ax2 = fig.add_subplot(specs[1,0]) # Second row and First column
    ax3 = fig.add_subplot(specs[1,1]) # Second row and Column
    
    by_year = df[['id', 'year']].groupby('year').sum().reset_index()
    ax1.bar(by_year['year'], by_year['id'])
    
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    ax2.plot(by_day['date'], by_day['id'])
    ax2.set_title('title: Avg Price by Day')
    
    by_week_of_year = df[['id', 'year_week']].groupby('year_week').mean().reset_index()
    ax3.plot(by_week_of_year['year_week'], by_week_of_year['id'])
    ax3.set_title('title: Avg Price by week Of Year')
    plt.xticks(rotation=60);
    


In [25]:
widgets.interactive(update_map, data = fixed(data), limit = date_limit)

interactive(children=(SelectionSlider(continuous_update=False, description='Disponivel', index=212, options=('…

# NOVAS PERGUNTAS DO CEO

# 0.0 Import Library

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt
from matplotlib import gridspec 
import ipywidgets as widgets
from ipywidgets import fixed
import plotly.express as px

## 0.1. Loading Data

In [5]:
data = pd.read_csv('kc_house_data.csv')

###                 1.Qual a quantidade de imóveis por nível?
     - Nivel 0: Preço entre R$ 0.00 e R$ 321.950
     - Nível 1: Preço entre R$ 321.950 e R$ 450.00
     - Nível 2: Preço entre R$ 450.00 e R$ 645.00
     - Nível 3: Preço acima de R$ 645.00

In [6]:
data['nivel'] = 'NA'
for i in range(len(data)):
    if (data.loc[i, 'price']>0) & (data.loc[i,'price'] < 321950):
        data.loc[i, 'nivel'] = 'nivel_0'
            
    elif (data.loc[i, 'price'] >= 321950 ) & (data.loc[i, 'price'] < 450000):
        data.loc[i, 'nivel'] = 'nivel_1'
        
    elif (data.loc[i, 'price'] >= 450000) & (data.loc[i, 'price'] < 645000):
        data.loc[i, 'nivel'] = 'nivel_2'
        
    else:
        data.loc[i, 'nivel'] = 'nivel_3'

### 2.Qual é a média do tamanho da sala de estar dos imóveis por 'size'?
     - Size 0: Tamanho entre  0 e  1427 sqft
     - Size 1: Tamanho entre  1427 e 1910 sqft
     - Size 2: Tamanho entre  1910 e 2550 sqft
     - Size 3: Tamanho acima de 2550 sqft

In [7]:
data['size'] = 'NA'
for i in range(len(data)):
    
    if (data.loc[i, 'sqft_living'] > 0) & (data.loc[i, 'sqft_living']<1427):
        data.loc[i, 'size'] = 'size_0'
    
    elif (data.loc[i, 'sqft_living'] >= 1427) & (data.loc[i, 'sqft_living']<1910):
        data.loc[i, 'size'] = 'size_1'
        
    elif (data.loc[i, 'sqft_living'] >= 1910) & (data.loc[i, 'sqft_living']<2550):
        data.loc[i, 'size'] = 'size_2'
        
    else:
        data.loc[i, 'size'] = 'size_3'        

### 3.Adicione as seguintes informações ao conjunto de dados originais:
    - Place Id: identificação da localização
    - OSM Type: Open Street Map type
    - Country: Nome do País
    - Country Code: Código do País

In [8]:
from geopy.geocoders import Nominatim

# Create empty rows
data['place_id'] = 'NA'
data['osm_type'] = 'NA'
data['country'] = 'NA'
data['country_code'] = 'NA'

geolocator = Nominatim(user_agent = 'geoapiExercises')

for i in range(len(data)):
    print('Loop:{} / {}'.format(i, len(data)))

# make query
query = str(data.loc[i, 'lat']) +','+ str(data.loc[i, 'long'])
# API Request
response = geolocator.reverse(query)

# Populate data

if 'place_id' in response.raw['address']:
    data.loc[i, 'place_id'] = response.raw['address']['place_id']
    
if 'osm_type' in response.raw['address']:
    data.loc[i, 'osm_type'] = response.raw['address']['osm_type']
    
if 'country' in response.raw['address']:
    data.loc[i, 'country'] = response.raw['address']['country']
    
if 'contry_code' in response.raw['address']:
    data.loc[i,'country_code'] = response.raw['address']['country_code']

Loop:0 / 21613
Loop:1 / 21613
Loop:2 / 21613
Loop:3 / 21613
Loop:4 / 21613
Loop:5 / 21613
Loop:6 / 21613
Loop:7 / 21613
Loop:8 / 21613
Loop:9 / 21613
Loop:10 / 21613
Loop:11 / 21613
Loop:12 / 21613
Loop:13 / 21613
Loop:14 / 21613
Loop:15 / 21613
Loop:16 / 21613
Loop:17 / 21613
Loop:18 / 21613
Loop:19 / 21613
Loop:20 / 21613
Loop:21 / 21613
Loop:22 / 21613
Loop:23 / 21613
Loop:24 / 21613
Loop:25 / 21613
Loop:26 / 21613
Loop:27 / 21613
Loop:28 / 21613
Loop:29 / 21613
Loop:30 / 21613
Loop:31 / 21613
Loop:32 / 21613
Loop:33 / 21613
Loop:34 / 21613
Loop:35 / 21613
Loop:36 / 21613
Loop:37 / 21613
Loop:38 / 21613
Loop:39 / 21613
Loop:40 / 21613
Loop:41 / 21613
Loop:42 / 21613
Loop:43 / 21613
Loop:44 / 21613
Loop:45 / 21613
Loop:46 / 21613
Loop:47 / 21613
Loop:48 / 21613
Loop:49 / 21613
Loop:50 / 21613
Loop:51 / 21613
Loop:52 / 21613
Loop:53 / 21613
Loop:54 / 21613
Loop:55 / 21613
Loop:56 / 21613
Loop:57 / 21613
Loop:58 / 21613
Loop:59 / 21613
Loop:60 / 21613
Loop:61 / 21613
Loop:62 / 21613
Lo

Loop:1378 / 21613
Loop:1379 / 21613
Loop:1380 / 21613
Loop:1381 / 21613
Loop:1382 / 21613
Loop:1383 / 21613
Loop:1384 / 21613
Loop:1385 / 21613
Loop:1386 / 21613
Loop:1387 / 21613
Loop:1388 / 21613
Loop:1389 / 21613
Loop:1390 / 21613
Loop:1391 / 21613
Loop:1392 / 21613
Loop:1393 / 21613
Loop:1394 / 21613
Loop:1395 / 21613
Loop:1396 / 21613
Loop:1397 / 21613
Loop:1398 / 21613
Loop:1399 / 21613
Loop:1400 / 21613
Loop:1401 / 21613
Loop:1402 / 21613
Loop:1403 / 21613
Loop:1404 / 21613
Loop:1405 / 21613
Loop:1406 / 21613
Loop:1407 / 21613
Loop:1408 / 21613
Loop:1409 / 21613
Loop:1410 / 21613
Loop:1411 / 21613
Loop:1412 / 21613
Loop:1413 / 21613
Loop:1414 / 21613
Loop:1415 / 21613
Loop:1416 / 21613
Loop:1417 / 21613
Loop:1418 / 21613
Loop:1419 / 21613
Loop:1420 / 21613
Loop:1421 / 21613
Loop:1422 / 21613
Loop:1423 / 21613
Loop:1424 / 21613
Loop:1425 / 21613
Loop:1426 / 21613
Loop:1427 / 21613
Loop:1428 / 21613
Loop:1429 / 21613
Loop:1430 / 21613
Loop:1431 / 21613
Loop:1432 / 21613
Loop:1433 

Loop:1936 / 21613
Loop:1937 / 21613
Loop:1938 / 21613
Loop:1939 / 21613
Loop:1940 / 21613
Loop:1941 / 21613
Loop:1942 / 21613
Loop:1943 / 21613
Loop:1944 / 21613
Loop:1945 / 21613
Loop:1946 / 21613
Loop:1947 / 21613
Loop:1948 / 21613
Loop:1949 / 21613
Loop:1950 / 21613
Loop:1951 / 21613
Loop:1952 / 21613
Loop:1953 / 21613
Loop:1954 / 21613
Loop:1955 / 21613
Loop:1956 / 21613
Loop:1957 / 21613
Loop:1958 / 21613
Loop:1959 / 21613
Loop:1960 / 21613
Loop:1961 / 21613
Loop:1962 / 21613
Loop:1963 / 21613
Loop:1964 / 21613
Loop:1965 / 21613
Loop:1966 / 21613
Loop:1967 / 21613
Loop:1968 / 21613
Loop:1969 / 21613
Loop:1970 / 21613
Loop:1971 / 21613
Loop:1972 / 21613
Loop:1973 / 21613
Loop:1974 / 21613
Loop:1975 / 21613
Loop:1976 / 21613
Loop:1977 / 21613
Loop:1978 / 21613
Loop:1979 / 21613
Loop:1980 / 21613
Loop:1981 / 21613
Loop:1982 / 21613
Loop:1983 / 21613
Loop:1984 / 21613
Loop:1985 / 21613
Loop:1986 / 21613
Loop:1987 / 21613
Loop:1988 / 21613
Loop:1989 / 21613
Loop:1990 / 21613
Loop:1991 

Loop:2935 / 21613
Loop:2936 / 21613
Loop:2937 / 21613
Loop:2938 / 21613
Loop:2939 / 21613
Loop:2940 / 21613
Loop:2941 / 21613
Loop:2942 / 21613
Loop:2943 / 21613
Loop:2944 / 21613
Loop:2945 / 21613
Loop:2946 / 21613
Loop:2947 / 21613
Loop:2948 / 21613
Loop:2949 / 21613
Loop:2950 / 21613
Loop:2951 / 21613
Loop:2952 / 21613
Loop:2953 / 21613
Loop:2954 / 21613
Loop:2955 / 21613
Loop:2956 / 21613
Loop:2957 / 21613
Loop:2958 / 21613
Loop:2959 / 21613
Loop:2960 / 21613
Loop:2961 / 21613
Loop:2962 / 21613
Loop:2963 / 21613
Loop:2964 / 21613
Loop:2965 / 21613
Loop:2966 / 21613
Loop:2967 / 21613
Loop:2968 / 21613
Loop:2969 / 21613
Loop:2970 / 21613
Loop:2971 / 21613
Loop:2972 / 21613
Loop:2973 / 21613
Loop:2974 / 21613
Loop:2975 / 21613
Loop:2976 / 21613
Loop:2977 / 21613
Loop:2978 / 21613
Loop:2979 / 21613
Loop:2980 / 21613
Loop:2981 / 21613
Loop:2982 / 21613
Loop:2983 / 21613
Loop:2984 / 21613
Loop:2985 / 21613
Loop:2986 / 21613
Loop:2987 / 21613
Loop:2988 / 21613
Loop:2989 / 21613
Loop:2990 

Loop:4399 / 21613
Loop:4400 / 21613
Loop:4401 / 21613
Loop:4402 / 21613
Loop:4403 / 21613
Loop:4404 / 21613
Loop:4405 / 21613
Loop:4406 / 21613
Loop:4407 / 21613
Loop:4408 / 21613
Loop:4409 / 21613
Loop:4410 / 21613
Loop:4411 / 21613
Loop:4412 / 21613
Loop:4413 / 21613
Loop:4414 / 21613
Loop:4415 / 21613
Loop:4416 / 21613
Loop:4417 / 21613
Loop:4418 / 21613
Loop:4419 / 21613
Loop:4420 / 21613
Loop:4421 / 21613
Loop:4422 / 21613
Loop:4423 / 21613
Loop:4424 / 21613
Loop:4425 / 21613
Loop:4426 / 21613
Loop:4427 / 21613
Loop:4428 / 21613
Loop:4429 / 21613
Loop:4430 / 21613
Loop:4431 / 21613
Loop:4432 / 21613
Loop:4433 / 21613
Loop:4434 / 21613
Loop:4435 / 21613
Loop:4436 / 21613
Loop:4437 / 21613
Loop:4438 / 21613
Loop:4439 / 21613
Loop:4440 / 21613
Loop:4441 / 21613
Loop:4442 / 21613
Loop:4443 / 21613
Loop:4444 / 21613
Loop:4445 / 21613
Loop:4446 / 21613
Loop:4447 / 21613
Loop:4448 / 21613
Loop:4449 / 21613
Loop:4450 / 21613
Loop:4451 / 21613
Loop:4452 / 21613
Loop:4453 / 21613
Loop:4454 

...

Loop:17064 / 21613
Loop:17065 / 21613
Loop:17066 / 21613
Loop:17067 / 21613
Loop:17068 / 21613
Loop:17069 / 21613
Loop:17070 / 21613
Loop:17071 / 21613
Loop:17072 / 21613
Loop:17073 / 21613
Loop:17074 / 21613
Loop:17075 / 21613
Loop:17076 / 21613
Loop:17077 / 21613
Loop:17078 / 21613
Loop:17079 / 21613
Loop:17080 / 21613
Loop:17081 / 21613
Loop:17082 / 21613
Loop:17083 / 21613
Loop:17084 / 21613
Loop:17085 / 21613
Loop:17086 / 21613
Loop:17087 / 21613
Loop:17088 / 21613
Loop:17089 / 21613
Loop:17090 / 21613
Loop:17091 / 21613
Loop:17092 / 21613
Loop:17093 / 21613
Loop:17094 / 21613
Loop:17095 / 21613
Loop:17096 / 21613
Loop:17097 / 21613
Loop:17098 / 21613
Loop:17099 / 21613
Loop:17100 / 21613
Loop:17101 / 21613
Loop:17102 / 21613
Loop:17103 / 21613
Loop:17104 / 21613
Loop:17105 / 21613
Loop:17106 / 21613
Loop:17107 / 21613
Loop:17108 / 21613
Loop:17109 / 21613
Loop:17110 / 21613
Loop:17111 / 21613
Loop:17112 / 21613
Loop:17113 / 21613
Loop:17114 / 21613
Loop:17115 / 21613
Loop:17116 /

Loop:17631 / 21613
Loop:17632 / 21613
Loop:17633 / 21613
Loop:17634 / 21613
Loop:17635 / 21613
Loop:17636 / 21613
Loop:17637 / 21613
Loop:17638 / 21613
Loop:17639 / 21613
Loop:17640 / 21613
Loop:17641 / 21613
Loop:17642 / 21613
Loop:17643 / 21613
Loop:17644 / 21613
Loop:17645 / 21613
Loop:17646 / 21613
Loop:17647 / 21613
Loop:17648 / 21613
Loop:17649 / 21613
Loop:17650 / 21613
Loop:17651 / 21613
Loop:17652 / 21613
Loop:17653 / 21613
Loop:17654 / 21613
Loop:17655 / 21613
Loop:17656 / 21613
Loop:17657 / 21613
Loop:17658 / 21613
Loop:17659 / 21613
Loop:17660 / 21613
Loop:17661 / 21613
Loop:17662 / 21613
Loop:17663 / 21613
Loop:17664 / 21613
Loop:17665 / 21613
Loop:17666 / 21613
Loop:17667 / 21613
Loop:17668 / 21613
Loop:17669 / 21613
Loop:17670 / 21613
Loop:17671 / 21613
Loop:17672 / 21613
Loop:17673 / 21613
Loop:17674 / 21613
Loop:17675 / 21613
Loop:17676 / 21613
Loop:17677 / 21613
Loop:17678 / 21613
Loop:17679 / 21613
Loop:17680 / 21613
Loop:17681 / 21613
Loop:17682 / 21613
Loop:17683 /

Loop:18363 / 21613
Loop:18364 / 21613
Loop:18365 / 21613
Loop:18366 / 21613
Loop:18367 / 21613
Loop:18368 / 21613
Loop:18369 / 21613
Loop:18370 / 21613
Loop:18371 / 21613
Loop:18372 / 21613
Loop:18373 / 21613
Loop:18374 / 21613
Loop:18375 / 21613
Loop:18376 / 21613
Loop:18377 / 21613
Loop:18378 / 21613
Loop:18379 / 21613
Loop:18380 / 21613
Loop:18381 / 21613
Loop:18382 / 21613
Loop:18383 / 21613
Loop:18384 / 21613
Loop:18385 / 21613
Loop:18386 / 21613
Loop:18387 / 21613
Loop:18388 / 21613
Loop:18389 / 21613
Loop:18390 / 21613
Loop:18391 / 21613
Loop:18392 / 21613
Loop:18393 / 21613
Loop:18394 / 21613
Loop:18395 / 21613
Loop:18396 / 21613
Loop:18397 / 21613
Loop:18398 / 21613
Loop:18399 / 21613
Loop:18400 / 21613
Loop:18401 / 21613
Loop:18402 / 21613
Loop:18403 / 21613
Loop:18404 / 21613
Loop:18405 / 21613
Loop:18406 / 21613
Loop:18407 / 21613
Loop:18408 / 21613
Loop:18409 / 21613
Loop:18410 / 21613
Loop:18411 / 21613
Loop:18412 / 21613
Loop:18413 / 21613
Loop:18414 / 21613
Loop:18415 /

Loop:19428 / 21613
Loop:19429 / 21613
Loop:19430 / 21613
Loop:19431 / 21613
Loop:19432 / 21613
Loop:19433 / 21613
Loop:19434 / 21613
Loop:19435 / 21613
Loop:19436 / 21613
Loop:19437 / 21613
Loop:19438 / 21613
Loop:19439 / 21613
Loop:19440 / 21613
Loop:19441 / 21613
Loop:19442 / 21613
Loop:19443 / 21613
Loop:19444 / 21613
Loop:19445 / 21613
Loop:19446 / 21613
Loop:19447 / 21613
Loop:19448 / 21613
Loop:19449 / 21613
Loop:19450 / 21613
Loop:19451 / 21613
Loop:19452 / 21613
Loop:19453 / 21613
Loop:19454 / 21613
Loop:19455 / 21613
Loop:19456 / 21613
Loop:19457 / 21613
Loop:19458 / 21613
Loop:19459 / 21613
Loop:19460 / 21613
Loop:19461 / 21613
Loop:19462 / 21613
Loop:19463 / 21613
Loop:19464 / 21613
Loop:19465 / 21613
Loop:19466 / 21613
Loop:19467 / 21613
Loop:19468 / 21613
Loop:19469 / 21613
Loop:19470 / 21613
Loop:19471 / 21613
Loop:19472 / 21613
Loop:19473 / 21613
Loop:19474 / 21613
Loop:19475 / 21613
Loop:19476 / 21613
Loop:19477 / 21613
Loop:19478 / 21613
Loop:19479 / 21613
Loop:19480 /

Loop:20427 / 21613
Loop:20428 / 21613
Loop:20429 / 21613
Loop:20430 / 21613
Loop:20431 / 21613
Loop:20432 / 21613
Loop:20433 / 21613
Loop:20434 / 21613
Loop:20435 / 21613
Loop:20436 / 21613
Loop:20437 / 21613
Loop:20438 / 21613
Loop:20439 / 21613
Loop:20440 / 21613
Loop:20441 / 21613
Loop:20442 / 21613
Loop:20443 / 21613
Loop:20444 / 21613
Loop:20445 / 21613
Loop:20446 / 21613
Loop:20447 / 21613
Loop:20448 / 21613
Loop:20449 / 21613
Loop:20450 / 21613
Loop:20451 / 21613
Loop:20452 / 21613
Loop:20453 / 21613
Loop:20454 / 21613
Loop:20455 / 21613
Loop:20456 / 21613
Loop:20457 / 21613
Loop:20458 / 21613
Loop:20459 / 21613
Loop:20460 / 21613
Loop:20461 / 21613
Loop:20462 / 21613
Loop:20463 / 21613
Loop:20464 / 21613
Loop:20465 / 21613
Loop:20466 / 21613
Loop:20467 / 21613
Loop:20468 / 21613
Loop:20469 / 21613
Loop:20470 / 21613
Loop:20471 / 21613
Loop:20472 / 21613
Loop:20473 / 21613
Loop:20474 / 21613
Loop:20475 / 21613
Loop:20476 / 21613
Loop:20477 / 21613
Loop:20478 / 21613
Loop:20479 /

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent='geopyExercises')

query = '47.5112, -122.257'
response = geolocator.reverse(query)

In [ ]:
import time 
from multiprocessing import Pool

In [ ]:
data['query'] = data[['lat', 'long']].apply(lambda x: str(x['lat'])+','+str(x['long']), axis = 1)

In [ ]:
def get_data(x):
    index, row = x
    time.sleep(1)
    
    #chamada API
    response = geolocator.reverse(row['query'])
    
    place_id = response.raw['place_id']
    osm_type = response.raw['osm_type']
    country = response.raw['address']['country']
    country_code = response.raw['address']['country_code']
    
    return place_id, osm_type, country, country_code

In [26]:
import defs

df1 = data[['id', 'query']].sample(10)

p = Pool(3)

start =time.process_time()
df1[['place_id', 'osm_type', 'country', 'country_code']] = p.map(defs.get_data, df1.iterrows())
end = time.process_time()

print('Time Elapsed: {}', end-start)

ModuleNotFoundError: No module named 'defs'

### 4.Adicione os seguintes filtros no mapa:
    - Tamanho mínimo da área da sala de estar.
    - Número mínimo de banheiros.
    - Valor máximo do preço.
    - Tamanho máximo da área do porão.
    - Filtro das condições do imóvel.
    - Filtro por ano de construção.

In [9]:
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

# define level of prices

data['level'] = data['price'].apply( lambda x: 0 if x <= 321950 
                                    else 1 if ( x > 321950 ) & ( x <= 450000)
                                    else 2 if ( x > 450000 ) & ( x <= 645000) else 3 )
data['level'] = data['level'].astype( int )

# Iteractive buttons

price_limit = widgets.IntSlider( 
    value = 540000,
    min = 75000,
    max = 7700000,
    step = 1,
    description='Maximun Price',
    disable=False,
    style={'description_width': 'initial'}
    )

waterfront_bar = widgets.Dropdown(
    options = data['is_waterfront'].unique().tolist(),
    value = 'yes',
    description = 'Water View',
    disable=False
    )

livingroom_limit = widgets.IntSlider( 
    value = int( data['sqft_living'].mean() ),
    min = data['sqft_living'].min(),
    max = data['sqft_living'].max(),
    step = 1,
    description='Minimum Living Room Size',
    disable=False,
    style={'description_width': 'initial'}
    )

bathroom_limit = widgets.IntSlider(
    value = int( data['bathrooms'].mean() ),
    min = data['bathrooms'].min(),
    max = data['bathrooms'].max(),
    step = 1,
    description='Minimum Bathroom Number',
    disable=False,
    style={'description_width': 'initial'}
    )
basement_limit = widgets.IntSlider(
    value = int( data['sqft_basement'].mean() ),
    min = data['sqft_basement'].min(),
    max = data['sqft_basement'].max(),
    step = 1,
    description='Minimum Basement Size',
    disable=False,
    style={'description_width': 'initial'}
    )
condition_limit = widgets.IntSlider(
    value = int( data['condition'].mean() ),
    min = data['condition'].min(),
    max = data['condition'].max(),
    step = 1,
    description='Minimum condition',
    disable=False,
    style={'description_width': 'initial'}
    )
year_limit = widgets.IntSlider(
    value = int( data['yr_built'].mean() ),
    min = data['yr_built'].min(),
    max = data['yr_built'].max(),
    step = 1,
    description='Year Built',
    disable=False,
    style={'description_width': 'initial'}
    )


In [10]:
def update_map( df, waterfront, limit, livingroom_limit, bathroom_limit,basement_limit, condition_limit, year_limit ):
    houses = df[(df['price'] <= limit) &
            (df['is_waterfront'] == waterfront) &
            (df['sqft_living'] >= livingroom_limit) &
            (df['bathrooms'] >= bathroom_limit) &
            (df['sqft_basement'] >= basement_limit) &
            (df['condition'] >= condition_limit) &
            (df['yr_built'] >= year_limit )][['id', 'lat', 'long', 'price','level']]
        
    fig = px.scatter_mapbox( houses,
                             lat="lat",
                             lon="long",
                             size="price",
                             color="level",
                             color_continuous_scale=px.colors.cyclical.IceFire,
                             size_max=15,
                             zoom=10)
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(height=600, margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()


In [11]:
widgets.interactive( update_map,
                    df=fixed( data ),
                    waterfront=waterfront_bar,
                    limit=price_limit,
                    livingroom_limit=livingroom_limit,
                    bathroom_limit=bathroom_limit,
                    basement_limit=basement_limit,
                    condition_limit=condition_limit,
                    year_limit=year_limit
                    )

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

### 5.Adicione os seguintes filtros no DashBoard:
    - Filtro por data disponivel para compra.
    - Filtro por ano de renovação.
    - Filtro se possui vista para a água ou não.

In [22]:
# Change data format
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

# Filtering

date_limit = widgets.SelectionSlider(
    options = data['date'].sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Max Available Date',
    disable = False,
    orientation = 'horizontal',
    style = {'description_width': 'initial'},
    readout = True
)

#Max year Renovated
year_limit = widgets.SelectionSlider(
    options = data['yr_renovated'].sort_values().unique().tolist(),
    value = 2000,
    description = 'Max Year',
    disable = False,
    continuous_update = False,
    orientation = 'horizontal',
    style = {'description_width': 'initial'},
    readout = True
)

# Waterfront
waterfront_limit = widgets.Checkbox(
    value =False,
    description = 'Waterfront?',
    disabled = False,
    indent = False
)

def update_map(data, date_limit, year_limit, waterfront_limit):
    #Filter data
    df = data[(data['date']<=date_limit) &
             (data['yr_renovated']>= year_limit) &
             (data['waterfront'] == waterfront_limit)].copy()

    fig = plt.figure(figsize = (24,12))
    specs = gridspec.GridSpec(ncols = 2, nrows = 2, figure = fig)
    
    ax1 = fig.add_subplot(specs[0,:]) # First Row
    ax2 = fig.add_subplot(specs[1,0]) # Second Row and First Column
    ax3 = fig.add_subplot(specs[1,1]) # Second Row and Second Column
    
    by_year = df[['id', 'year']].groupby('year').sum().reset_index()
    sns.barplot(x ='year', y ='id', data = by_year, ax = ax1)
    
    # First Graph
    by_day = df[['id', 'date']].groupby('date').mean().reset_index()
    sns.lineplot(x = 'date', y = 'id', data = by_day, ax = ax2)
    ax2.set_title('Title: Avg Price by Day')
    
    df['year_week'] = pd.to_datetime(df['date']).dt.strftime('%Y-%U')
    by_week_of_year = df[['id', 'year_week']].groupby('year_week').mean().reset_index()
    sns.barplot(x = 'year_week', y = 'id', data = by_week_of_year, ax = ax3)
    ax3.set_title('Title: Avg Price by week of Year')
    plt.xticks(rotation=60);
    

In [23]:
widgets.interactive(update_map,
                   data = fixed(data),
                   date_limit = date_limit,
                   year_limit = year_limit,
                   waterfront_limit = waterfront_limit)

interactive(children=(SelectionSlider(description='Max Available Date', index=212, options=('2014-05-02', '201…